In [ ]:
# hide
# default_exp core
# all_slow

# fastcgi API

> API details for fastcgi

In [ ]:
#export
from fastcore.foundation import *
from fastcore.utils import *
from fastcore.meta import *

import struct
from socketserver import StreamRequestHandler,BaseRequestHandler,UnixStreamServer,TCPServer
from enum import Enum
from io import BytesIO,TextIOWrapper

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *
import subprocess,time

This library follows the [FastCGI spec](http://www.mit.edu/~yandros/doc/specs/fcgi-spec.html). It only supports the *Responder* role, and does not support multiplexing (which is not supported by any servers, so is unlikely to be an issue).

## Enums

In [ ]:
#export
Record = Enum('Record', 'BEGIN_REQUEST ABORT_REQUEST END_REQUEST PARAMS STDIN '
               'STDOUT STDERR DATA GET_VALUES GET_VALUES_RESULT')
Role = Enum('Role', 'RESPONDER AUTHORIZER FILTER')
Status = Enum('Status', 'REQUEST_COMPLETE CANT_MPX_CONN OVERLOADED UNKNOWN_ROLE')

These `enum`s are used throughout the library, and have the same meanings as the `FCGI_` constants `#define`d in the spec.

In [ ]:
for o in Record,Role,Status: print(list(o))

[<Record.BEGIN_REQUEST: 1>, <Record.ABORT_REQUEST: 2>, <Record.END_REQUEST: 3>, <Record.PARAMS: 4>, <Record.STDIN: 5>, <Record.STDOUT: 6>, <Record.STDERR: 7>, <Record.DATA: 8>, <Record.GET_VALUES: 9>, <Record.GET_VALUES_RESULT: 10>]
[<Role.RESPONDER: 1>, <Role.AUTHORIZER: 2>, <Role.FILTER: 3>]
[<Status.REQUEST_COMPLETE: 1>, <Status.CANT_MPX_CONN: 2>, <Status.OVERLOADED: 3>, <Status.UNKNOWN_ROLE: 4>]


In [ ]:
#export
def _S(fmt): return struct.Struct('!'+fmt) # use `struct` "network order"
_rec_struct,_endreq_struct,_begreq_struct,_long_struct = _S('BBHHbb'),_S('LBxxx'),_S('Hb5s'),_S('L')
_streams_data = Record.STDIN,Record.DATA
_streams_in  = (Record.PARAMS,) + _streams_data
_streams = _streams_in + (Record.STDOUT,Record.STDERR,Record.END_REQUEST)

In [ ]:
#export
def readlen(r):
    "Read the length of the next fcgi parameter"
    # See http://www.mit.edu/~yandros/doc/specs/fcgi-spec.html#S3.4
    a = r(1)
    res = a[0]
    if res>>7: res =_long_struct.unpack(a+r(3))[0] & 0x7fffffff
    return res

In [ ]:
t = 1_000_000_101
s = struct.pack('!L', t | (1<<31))
test_eq(readlen(BytesIO(s).read), t)

In [ ]:
def _recv_struct(recv, fmt):
    if not isinstance(fmt,struct.Struct): fmt = _S(fmt)
    res = fmt.unpack(recv(fmt.size))
    return res[0] if len(res)==1 else res

In [ ]:
def recv_record(r):
    _,typ,_,contentlen,padlen,_ = _recv_struct(r, _rec_struct)
    c = _recv_struct(r, f'{contentlen}s{"x"*padlen}')
    typ = Record(typ)
    if   typ==Record.BEGIN_REQUEST: c = _begreq_struct.unpack(c)[:2]
    elif typ==Record.ABORT_REQUEST: c = ()
    return typ,c

In [ ]:
#hide
d = b'\x01\x01\x00\x01\x00\x08\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x01\x04\x00\x01\x01\xea\x06\x00\x12\x10REQUEST_TIME_FLOAT1605811743685432\x0f\tPATH_TRANSLATED/setup.py\t\tPATH_INFO/setup.py\x13\x01HTTP_CONTENT_LENGTH4\x0e\x04REQUEST_METHODPOST\x0b\x03REMOTE_ADDR::1\x0b\x05REMOTE_PORT52102\x0b\tSCRIPT_NAME/setup.py\x0e\tORIG_PATH_INFO/setup.py\x0b\x03HTTP_ACCEPT*/*\x0f\x08SERVER_PROTOCOLHTTP/1.1\x0c\nREQUEST_TIME1605811743\x0c\x03QUERY_STRINGa=1\t\x0eHTTP_HOSTlocalhost:6065\x11\x10HTTP_CONTENT_TYPEapplication/json\x0f\tSERVER_SOFTWAREhttp2fcgi\r\tDOCUMENT_ROOT/setup.py\x0f\tSCRIPT_FILENAME/setup.py\x0b\rREQUEST_URI/setup.py?a=1\x0b\x00AUTH_DIGEST\x0f\x0bHTTP_USER_AGENTcurl/7.71.1\x00\x00\x00\x00\x00\x00\x01\x04\x00\x01\x00\x00\x00\x00\x01\x05\x00\x01\x00\x04\x04\x00test\x00\x00\x00\x00\x01\x05\x00\x01\x00\x00\x00\x00'

In [ ]:
b = BytesIO(d).read
recv_record(b)

typ,p = recv_record(b); typ,p

(<Record.PARAMS: 4>,
 b'\x12\x10REQUEST_TIME_FLOAT1605811743685432\x0f\tPATH_TRANSLATED/setup.py\t\tPATH_INFO/setup.py\x13\x01HTTP_CONTENT_LENGTH4\x0e\x04REQUEST_METHODPOST\x0b\x03REMOTE_ADDR::1\x0b\x05REMOTE_PORT52102\x0b\tSCRIPT_NAME/setup.py\x0e\tORIG_PATH_INFO/setup.py\x0b\x03HTTP_ACCEPT*/*\x0f\x08SERVER_PROTOCOLHTTP/1.1\x0c\nREQUEST_TIME1605811743\x0c\x03QUERY_STRINGa=1\t\x0eHTTP_HOSTlocalhost:6065\x11\x10HTTP_CONTENT_TYPEapplication/json\x0f\tSERVER_SOFTWAREhttp2fcgi\r\tDOCUMENT_ROOT/setup.py\x0f\tSCRIPT_FILENAME/setup.py\x0b\rREQUEST_URI/setup.py?a=1\x0b\x00AUTH_DIGEST\x0f\x0bHTTP_USER_AGENTcurl/7.71.1')

In [ ]:
recv_record(b)

(<Record.PARAMS: 4>, b'')

In [ ]:
def _params(s):
    b = BytesIO(s)
    r = b.read
    while b.tell()<len(s):
        lk,lv = readlen(r),readlen(r)
        yield _recv_struct(r, f'{lk}s{lv}s')

def params(s):
    return {k.decode():v.decode() for k,v in _params(s)}

In [ ]:
params(p)

{'REQUEST_TIME_FLOAT': '1605811743685432',
 'PATH_TRANSLATED': '/setup.py',
 'PATH_INFO': '/setup.py',
 'HTTP_CONTENT_LENGTH': '4',
 'REQUEST_METHOD': 'POST',
 'REMOTE_ADDR': '::1',
 'REMOTE_PORT': '52102',
 'SCRIPT_NAME': '/setup.py',
 'ORIG_PATH_INFO': '/setup.py',
 'HTTP_ACCEPT': '*/*',
 'SERVER_PROTOCOL': 'HTTP/1.1',
 'REQUEST_TIME': '1605811743',
 'QUERY_STRING': 'a=1',
 'HTTP_HOST': 'localhost:6065',
 'HTTP_CONTENT_TYPE': 'application/json',
 'SERVER_SOFTWARE': 'http2fcgi',
 'DOCUMENT_ROOT': '/setup.py',
 'SCRIPT_FILENAME': '/setup.py',
 'REQUEST_URI': '/setup.py?a=1',
 'AUTH_DIGEST': '',
 'HTTP_USER_AGENT': 'curl/7.71.1'}

In [ ]:
#export
def _send_content(r, typ, *args):
    "Send the content section of a fastcgi binary record of `typ` to `req`"
    if typ==Record.END_REQUEST: c = _endreq_struct.pack(*args)
    else: c = args[0] if args else b''
    r(_rec_struct.pack(1, typ.value, 1, len(c), 0, 0) + c)

In [ ]:
#export
class ByteStream(BytesIO):
    def __init__(self, typ:Record, r): self.typ,self.r = typ,r

    def _send(self, *args): _send_content(self.r, self.typ, *args)
    def send(self):
        for o in chunked(self.getvalue(), 2**15): self._send(bytes(o))
        self._send()

In [ ]:
#export
class FcgiHandler(StreamRequestHandler):
    "A request handler that processes FastCGI streams and parameters"
    def setup(self):
        super().setup()
        self.streams = L(_streams).map_dict(ByteStream, r=self.wfile.write)
        sz,self.length = 0,1
        while sz<self.length: sz += ifnone(self._recv(), 0)

    def finish(self):
        super().finish()
        self['stdout'].send()
        self['stderr'].send()
        self['end_request']._send(0, Status.REQUEST_COMPLETE.value)
        self['stdin'].seek(0)
    
    def _recv(self):
        typ,c = recv_record(self.rfile.read)
        if typ in _streams_in:
            self[typ].write(c)
            if typ==Record.PARAMS and not c:
                self.environ = params(self[typ].getbuffer())
                self.length = int(self.environ['HTTP_CONTENT_LENGTH'] or 0)
        if typ in _streams_data: return len(c)

    @property
    def content(self): return self['stdin'].getbuffer()
    def __getitem__(self,k): return self.streams[Record[k.upper()] if isinstance(k,str) else k]

This is used in much the same way as [StreamRequestHandler](https://docs.python.org/3/library/socketserver.html#request-handler-objects), except that receiving the data is handled for you before your `handle` method is called. All headers are available in the `params` dictionary. All streams are available through indexing (i.e. using `__getitem__`). The `stdin` stream contains the data sent to your handler. Write to the `stdout` and `stderr` streams to send data to the client, or use the `write` method.

In [ ]:
#export
@patch
def write(self:FcgiHandler, b:bytes, err=False):
    "Write `b` to stderr (if `err`) or stdout (otherwise)"
    self['stderr' if err else 'stdout'].write(b)

Here's an example subclass:

In [ ]:
class TestHandler(FcgiHandler):
    def handle(self):
        print('query:', self.environ['QUERY_STRING'])
        print('content type:', self.environ['HTTP_CONTENT_TYPE'])
        print('stdin:', self.content)
        self.write(b"Content-type: text/html\r\n\r\n<html>foobar</html>\r\n")

To test it, we'll use an http➡fcgi proxy. We can download `http2fcgi` and run it in the background as follows:

In [ ]:
run('./get_http2fcgi.sh')
proc = subprocess.Popen(['./http2fcgi'])

We can now test the handler by running a server in the background...

In [ ]:
p = Path('fcgi.sock')

@threaded
def _f():
    with UnixStreamServer(str(p), TestHandler) as srv: srv.handle_request()

if p.exists(): p.unlink()
t = _f()
time.sleep(0.2) # wait for server to start

...and use `curl` to test it:

In [ ]:
!curl 'http://localhost:6065/setup.py?a=1' -X POST -H "Content-Type: application/json" -d test

query: a=1
content type: application/json
stdin: <memory at 0x7f1b205b1a10>



Finally, we kill the `http2fcgi` background process.

In [ ]:
proc.terminate()

### Convenience methods

In [ ]:
#export
class _Wrapper(TextIOWrapper): close=TextIOWrapper.flush

def _print_bytes(s:str, stream):
    "Convert `s` to `bytes`, using `\r\n` for newlines"
    b = _Wrapper(stream, newline='\r\n', encoding='utf8')
    print(s, file=b)

In [ ]:
#export
@patch
def print(self:FcgiHandler,s=""):
    "Write a `str` to `self.stdout` as bytes, converting line endings to `\r\n`"
    _print_bytes(s, self['stdout'])

Instead of `self.stdout.write(...)` (which requires byte strings and `\r\n` line endings, and does not append a line ending automatically) we can use `print`:

```python
self.print("Content-type: text/html")
self.print()
self.print("<html>foobar</html>")
```

In [ ]:
#export
@patch
def err(self:FcgiHandler,s=""):
    "Write a `str` to `self.stderr` as bytes, converting line endings to `\r\n`"
    _print_bytes(s, self['stderr'])

For errors, you can either `write` to `stderr`:

```python
self.stderr.write(b"Something went wrong\r\n")
```

or call `error`, which is like `print`, but for `stderr`:

```python
self.error("Something went wrong")
```

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
